# Classification-1

まず、2値分類用のサンプルデータであるBreast Cancerを読み込みます。特徴量が全て数値データなので、欠損値の補完処理やカテゴリ変数のフラグ変数化が不要なデータであることを思い出しましょう。

In [1]:
# Load and return the breast cancer wisconsin dataset (classification).
# The breast cancer dataset is a classic and very easy binary classification dataset.
import pandas as pd
from sklearn.datasets import load_breast_cancer

dataset = load_breast_cancer()

# Set dataframe
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = pd.DataFrame(dataset.target, columns=['y'])

# check the shape
print('----------------------------------------------------------------------------------------')
print('X shape: (%i,%i)' %X.shape)
print('y shape: (%i,%i)' %y.shape)
print('----------------------------------------------------------------------------------------')
print(y.groupby('y').size())
print('y=0 means Marignant(悪性), y=1 means Benign(良性):')
print('----------------------------------------------------------------------------------------')
X.join(y).head()

----------------------------------------------------------------------------------------
X shape: (569,30)
y shape: (569,1)
----------------------------------------------------------------------------------------
y
0    212
1    357
dtype: int64
y=0 means Marignant(悪性), y=1 means Benign(良性):
----------------------------------------------------------------------------------------


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,y
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


さて、次にk近傍法とロジスティック回帰の呼び出し方を学びましょう。KNNは怠惰学習の一つで、sklearnでもneighborsという特別なクラスから呼び出しています。ロジスティック回帰は、重みベクトルと特徴量の一次線形結合をベースとしたアルゴリズムなので、線形モデルから呼び出せます。

In [2]:
# import libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

回帰でも学びましたが、機械学習のアルゴリズムは特徴量ベクトルを標準化するのが普通です。また、標準化処理とモデル学習（重みの最適化）のプロセスをひとまとめに記述できるパイプラインも合わせて呼び出しましょう。

In [3]:
# import libraries
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

回帰モデルの評価にはR2スコアなどがありました。分類モデルの評価はどのようにしたらよいでしょうか。最も代表的なものは、正解率（Accuracy）です。正解率は、予測対象データ（スコアリングデータ）の全件をNとした時、正例を正しく正例と分類できた数TP、負例を正しく負例と分類できた数TFとすると、 以下の数式で表せます。<br><center>Accuracy = (TP + TF) / N</center><br>つまり、正例・負例を問わず正しく分類されたサンプル数の割合です。sklearnでは正解データ（正解ベクトル）と予測データ（予測ベクトル）を与えると、Accuracyを計算してくれる関数が存在します。それを以下で呼び込みましょう。

In [4]:
# 分類モデルの評価指標（計算ライブラリ）の読み込み
# sklearn.metrics.accuracy_score(y_true, y_pred, normalize=True, sample_weight=None)
from sklearn.metrics import accuracy_score

それでは、回帰で学んだこと、これまでの内容を整理します。以下コードの各行の役割を理解できていることを確認して下さい。

In [5]:
# import basice apis
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# import Sample Data to learn models
dataset = load_breast_cancer()
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = pd.DataFrame(dataset.target, columns=['y'])

# Holdout
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20, random_state=1)

# set pipelines for two different algorithms
pipe_knn_5 = Pipeline([('scl',StandardScaler()),('est',KNeighborsClassifier())])
pipe_knn_50 = Pipeline([('scl',StandardScaler()),('est',KNeighborsClassifier(n_neighbors=50))])
pipe_logistic = Pipeline([('scl',StandardScaler()),('est',LogisticRegression(random_state=1))])

knn_5はデフォルトのK近傍法のパフォーマンス確認、pipe_knn_50は近傍数を増やしたときの性能確認用です。Kを増やすことで予測性能（正解率）がどのように変化するか考えてみましょう。pipe_logisticはロジスティック回帰の性能確認用です。kNNよりは良いだろうというのが通常の期待です。それでは各パイプラインを学習(fit)させましょう。正常終了すれば終了です。

In [6]:
b
pipe_knn_5.fit(X_train,y_train.as_matrix().ravel())
pipe_knn_50.fit(X_train,y_train.as_matrix().ravel())
pipe_logistic.fit(X_train,y_train.as_matrix().ravel())

# print pipelines
print(pipe_knn_5)
print(pipe_knn_50)
print(pipe_logistic)

Pipeline(memory=None,
     steps=[('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'))])
Pipeline(memory=None,
     steps=[('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=50, p=2,
           weights='uniform'))])
Pipeline(memory=None,
     steps=[('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])


最後に、３つのパイプラインの分類器としての性能（正解率）を比較します。まずpipelineのpredictの出力ベクトルを見て下さい。y_pred in {0,1}で予測ラベルが付与されていることがわかります。

In [7]:
pipe_logistic.predict(X_train)[:10]

array([0, 0, 1, 1, 1, 1, 1, 0, 0, 1])

今回は使いませんが、実務では予測ラベルの他に、各クラスに分類される予測確率をよく使います。確率を出力させたい場合は、predictをpredic_probaに変更するだけです。最初の列はy=0と分類される予測確率、2番目の列はy=1と分類される予測確率です。

In [8]:
pipe_logistic.predict_proba(X_train)[:10]

array([[  9.99441634e-01,   5.58366357e-04],
       [  9.99962234e-01,   3.77655428e-05],
       [  1.12299304e-06,   9.99998877e-01],
       [  4.10316174e-04,   9.99589684e-01],
       [  1.83025794e-04,   9.99816974e-01],
       [  7.88575120e-03,   9.92114249e-01],
       [  6.74588243e-02,   9.32541176e-01],
       [  9.99956558e-01,   4.34422063e-05],
       [  9.99999212e-01,   7.88384042e-07],
       [  2.32654465e-02,   9.76734554e-01]])

accuracy_scoreは第一引数に正解ラベル、第二引数に予測ラベル（確率ではない）を指定するので、今回はpredictで予測ラベルを与えましょう。

In [2]:
print('KNN(5)_Train: %.3f' % accuracy_score(y_train, pipe_knn_5.predict(X_train)))
print('KNN(5)_Test: %.3f' % accuracy_score(y_test, pipe_knn_5.predict(X_test)))

print('KNN(50)_Train: %.3f' % accuracy_score(y_train, pipe_knn_50.predict(X_train)))
print('KNN(50)_Test: %.3f' % accuracy_score(y_test, pipe_knn_50.predict(X_test)))

print('Logistic_Train: %.3f' % accuracy_score(y_train, pipe_logistic.predict(X_train)))
print('Logistic_Test: %.3f' % accuracy_score(y_test, pipe_logistic.predict(X_test)))

NameError: name 'accuracy_score' is not defined

k近傍法のkを5から50に増やしましたが、性能が落ちてしまいました。多くのデータを見過ぎて平均値予測に近づき過ぎてしまったようです。予測値が無難過ぎてTrainスコアとTestスコアが近いのも特徴です。ロジスティック回帰は、k近傍法より3pt（0.956 to 0.982）程度、改善することを確認できました。ただし、どのように改善するか（しないか）はデータ次第ですので、代表的なアルゴリズムをいつでも試せる準備をしておくことが、プロジェクト進行において大切です。